In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import timedelta

from requests_cache import install_cache
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting.probabilistic import PinballLoss
from sktime.split import temporal_train_test_split
from sktime.utils import plotting

from probafcst.data import get_bikes_data, get_energy_data, get_no2_data
from probafcst.utils.time import get_forecast_dates

In [ ]:
install_cache("test_cache", backend="sqlite", expire_after=timedelta(days=5))

In [ ]:
bikes = get_bikes_data(start_date="2010-01-01")
bikes.head()

In [ ]:
energy = get_energy_data()

In [ ]:
energy.info()

In [ ]:
no2 = get_no2_data(start_date="2010-01-01")

In [ ]:
no2.info()

In [ ]:
load = energy.copy()

In [ ]:
y_train, y_test = temporal_train_test_split(load, test_size=24 * 7)
fh = ForecastingHorizon(y_test.index, is_relative=False)
forecaster = NaiveForecaster(strategy="mean", sp=24)
# forecaster = StatsForecastAutoARIMA()
forecaster.fit(y_train)

quantile_levels = [0.025, 0.25, 0.5, 0.75, 0.975]
y_pred = forecaster.predict_quantiles(fh, alpha=quantile_levels)

In [ ]:
columns = [y_pred[i] for i in y_pred.columns]
fig, ax = plotting.plot_series(y_test, *columns)

In [ ]:
loss = PinballLoss(score_average=True)
loss(y_test, y_pred)

In [ ]:
# make a submission

timestamps, forecast_hours = get_forecast_dates(load.index[-1])

forecaster = NaiveForecaster()
forecaster.fit(load)
fh = ForecastingHorizon(forecast_hours, is_relative=True)
y_pred = forecaster.predict_quantiles(fh, alpha=quantile_levels)
y_pred

In [ ]:
from probafcst.utils import create_submission

create_submission(energy_preds=y_pred.values)